https://zablo.net/blog/post/pytorch-resnet-mnist-jupyter-notebook-2021/

DATA PREPARATION

Dataset Menggunakan MNIST yang memiliki 60.000 train set dan 10.000 test set. Setiap gambar memiliki ukuran 28 x 28 pixel dan 1 channel karena grayscale.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms
from torchvision.models import resnet50
import torch.nn as nn
import torchvision


def cv2_imshow(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()

    # plt.figure(figsize=(1, 1))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()



BATCH_SIZE = 64
NUM_CLASSES = 10
EPOCHS = 1
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.001
MOMENTUM = 0.9

# data augmentation
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.4914, 0.4822, 0.4465],
#         std=[0.2023, 0.1994, 0.2010])
# ])

transform = transforms.ToTensor()

# list of PIL format. selain itu, transform utk data augmentation
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True)

# DataLoader mengubah list of PIL format menjadi list of tensor
# selain itu, DataLoader memiliki fungsi-fungsi khusus lainnya
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True)

print("train dataset:", len(train_dataset))
print("test dataset :", len(test_dataset))
print("train loader :", len(train_loader))
print("test loader  :", len(test_loader))

dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))


In [58]:

model = resnet50(num_classes=NUM_CLASSES)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2),
                        padding=(3, 3), bias=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    momentum=MOMENTUM)

for epoch in range(EPOCHS):
    # tells model that this process are training mode
    model.train()

    # in an iteration, there are many images depend on specified BATCH_SIZE
    # ex: tensor.Size = [64, 1, 28, 28] for images dan [64] labels
    for batch, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        prediction = model(images)
        loss = loss_fn(prediction, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1 == 0:
            loss, current = loss.item(), batch * len(images)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(train_loader):>5d}]")

loss: 2.687032  [    0/  938]


KeyboardInterrupt: 